# Run this cell first

In [ ]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  %pip install git+https://github.com/abrown41/AutoFeedback
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  %pip install "git+https://github.com/autofeedback-exercises/exercises.git#subdirectory=New-SOR3012/Event_based_simulation"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass


# Introduction

This notebook will introduce you to a method called event-driven simulation, which can be used to simulate random processes.  Watch the following video before you start, which explains how we can arrive at a continuous analogue of the theory of Markov chains that we have discussed in the last two sets of exercises by using calculus.  We will be using ideas from theory of continuous time Markov chains when we build models using event-driven simulation

In [ ]:
%%HTML 
<iframe width="560" height="315" src="https://www.youtube.com/embed/tbA2DnKTRxM?si=A4QGlTQwwcwwOK0T" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

As always, run the following cell and load the libraries that we need before starting the exercises.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

# The exponential random variable

The simplest Markov example of a Markov chain in continuous time is a two-state chain that has a jump rate matrix that looks like this:

$$
\mathbf{Q} = \left(
\begin{matrix}
-\lambda & \lambda \\
0 & 0
\end{matrix}
\right)
$$

As discussed in the following video, this Markov chain can be used to model the amount of time it takes for an event to happen.

In [2]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/BdOB8x3SVAE?si=kPHPFyRriqPGFYv1" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

As discussed in the following video, you can generate an exponential random variable by first generating a uniform random variable in the usual way.  This random variable is inserted into the inverse function for the cumulative probability distribution for the exponential random variable.  The random variable that emerges from this inverse function is then a sample from an exponential random variable.

In [3]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/Ce_tWXEPTtY?si=agNemxqilcHTYlxs" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

This code for generating exponential random variables, is central to everything we will be doing in this notebook.  Your task in this exercise is thus to write a function called `exponential` that takes in a parameter `lam`.  This function should return an exponential random variable generated from a distribution with parameter `lam`.  Please note that you can calculate the natural logarithm of a variable, `x`, by using the command:

```python
logx = np.log(x)
```

as you will need this in your function to calculate the random variable.


In [ ]:
def exponential(lam):
  # Your code to generate an exponential random variable goes here


# Here is some code that just tests your function is working correctly
print( exponential(2), exponential(2), exponential(2), exponential(2) )


In [ ]:
runtest(['test_lam'])

# Histogram for continuous random variables

The exponential random variable that you have just learned to generate is an example of a continuous random variable.  Continuous random variables (unlike the discrete random variables that we have encountered in other parts of this module) have a probability density function rather than a probability mass function.  The probability density function for the random variable, $X$, is given by:

$$
f(x) = \frac{\textrm{d}P(X\le x)}{\textrm{d}x}
$$

where $P(X\le x)$ is the cumulative probability distribution function for the random variable.

In previous weeks, I have shown you how we can estimate the probablity mass function for a discrete random variable by building a histogram.  As explained in the following video, we can use a similar algorithm to estimate the probability density function.  The idea in this next exercise is to give you some practise at applying this technqiue. 

In [4]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/-aS_CrskEYE?si=PXtB28n_VCRHrNRy" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In the cell below I would like you to use these ideas to write code to estimate the probability density function for the exponential random variable.  You can do this by repeatedly sampling the random variable by calling the function that you wrote for generating it in the previous exercise multiple times.

There are two important things to bear in mind that are different when you calculate the histogram for a continuous random variable.  The first is the way that we use the `np.floor` function to describe the continuous interval into `n` discrete bins.
If we know that the random variable is between `min` and `max` then we can work out the widths of these bins using the following python code:

```python
delr = ( max - min ) / n
```

If we now generate a random variable `U` we can work out the bin that the random falls within using the following code:

```python
mybin = int ( ( U - min ) / delr )
```

We can thus use the above within the normal code that we would employ to compute the histogram. Now, however, the height of the bars in the unormalized histogram are equal to the number of random variable that fall into each of the line segments of interest.

As we are trying to estimate a probablity density function we must sure that the function is normalised correctly so that:

$$
\int_{-\infty}^\infty f(x) \textrm{d}x = 1
$$

So the area under your final histogram should be one.  When you normalise the points in the list called counts you should thus divide by the number of points sampled and the width of the bins (`delr`).
The estimates of the probability densities that once you have completed this procedure are plotted at the midpoints of the bins.  These midpoints can be generated using the command:

```python
for i in range(nbins) : xvals[i] = (i+0.5)*delr
```

Furthermore, the probablity density function should be plotted using a line rather than a series of bars as it is a continuous function.  The probablity mass function that we have estimated in previous exercises only has non-zero values for particular discrete values of x.

With all this in mind, for this exercise I want you to estimate the probablity density function for the exponential random variable by sampling.   The one small problem that we have is that the exponential random variable can in theory take any value between 0 and infinity.  The probability of having a very high value for the random variable is, however, very small.  We can thus safely truncate the range and calculate our estimate of the probability density function over some finite range.

Please note that you must set the variable called `lamd` equal to the parameter for the exponential random variable you sample in order to pass the tests for the task.  

In [ ]:
lamd = 2      # The parameter for the exponential random variable you should sample
xmax = 6     # The maximum value for x that you should use in your histogram
nbins = 200   # The number of bins you should use for your histogram
histo = np.zeros(nbins)     # The variable you should use to store the histogram
xvals = np.zeros(nbins)     # The midpoints of your histogram bins that are used for plotting

# Your code to compute an estimate for the probability density function of an exponential random
# variable goes here.


# This draws the histogram - do not delete this code
plt.plot( xvals, histo, 'k-')
plt.xlabel('Random variable value')
plt.ylabel('Probability')


# This code is required for the autofeedback- don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_plot'])

# The poisson process

Returning to the business of event-driven simulation, lets move on from having a random process to describe the occurance of a single event.  Let's now look at how we can introduce multiple events by considering the Poisson process that is described in the following video.

In [8]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/kWvG0_p4wO8?si=8X8MOyrMadY-hGb7" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Let's now try to sample the poisson process.  This turns out to be rather easy because, as is explained in the following video, the distribution for the times between adjacent events is exponential.  Consequently, we can generate a Poisson process by simply generating exponential random variables and adding them together.  The sum of three such exponential random variables will thus give us the time at which the third of the events occurs in our Poisson process.

In [7]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/dGsXWwGS7Go?si=7Pyuf1bje2zRXCSp" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Let's explore these ideas by writing some code.  To complete this code you must write a loop in which the function `exponential` is called multiple times.  Within this loop you will need to set the elements of the list called `arrival_times` and the elements of the list called `number_of_events`.  The first of these two lists should be set equal to the times at which the events occurred and the second of these two lists should be equal to the number of events that have occurred by that time.

The code that is already there will draw a graph with points that have their x-coordinates equal to `arrival_times` and their y-coordinates equal to `number_of_events`.  In other words, the final result is a graph showing the how the number of events that have occurred changes with time.

Notice that in order to pass the test the inter arrival times between the events in your Poisson process must be exponentially distributed random variables with the parameter `lamd`.

In [ ]:
nevents = 20
lamd = 2
number_of_events = np.zeros(nevents)
arrival_times = np.zeros(nevents)
for i in range(nevents) :
  # You need to write code in here in order to set the elements
  # of the two lists number_of_events and arrival_times


plt.plot( arrival_times, number_of_events, 'ko' )
plt.xlabel('time')
plt.ylabel('number of events')
# This code is required for the autofeedback- don't delete it!
fighand = plt.gca()



In [ ]:
runtest(['test_plot_1'])

# The Object Oriented Paradigm

In the next few exercises we are going to use these ideas that have been introduced above to simulate a queue. In our code for simulating a queue we are going to use a different method for organising programs, which is called the object oriented paradigm.  The ideas behind the object oriented paradigm are explained in the following video.  The examples in this video are in javascript but the essential idea is the same in python.

In [5]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/pTB0EiLXUC8?si=8la5Md3n-4P4Mxc7" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

# Simulating a queue

Having introduced the basic theory of Markov chains in continuous time by introducing the exponential random variable and the poisson process, lets now use these ideas to model a real world system - a queue.  

Queues form where a service is offered and, as is explained in the following video, there is an interplay between the random amount of time it takes to serve each customer and the random times between customer arrivals. 

In [9]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/TlHUxvCo3Sg?si=rzt1TQrNF542c5PU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

To simulate our queue we are going to use the object-oriented paradigm and the code discussed in the following video.  Watch the video and see if you can fill in the gaps in the outline for the simulation object in the code that follows.

In [6]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/oJyf8Q0KLRY?si=L2ufhv57s3633ODn" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Now try to copy the code from the video into the following cell.  Once you have done this complete the function called `average_time_in_queue`.  This function takes three variables in input:

* `t` the total time that you will run the simulation for.
* `lambd` the lambda parameter for the queue
* `mu` the mu parameter for the queue

It should return the average length of time a person spends in the queue in output.  The average time a person spends queuing or being served should be calculated as the total time people spent waiting divided by the number of departures.  If you run the simulation for long enough this will be approximately equal to the average wait time.

N.B.  I made a small change to the way he defined the class in the video.  I have setup the constructor (`__init__`) so that it takes the $\lambda$ and $\mu$ parameters for the queue in input.

In [1]:
class Simulation : 
    def __init__(self,lambd,mu) :
        self.lam = lambd
        self.mu = mu
        pass
    def advance_time(self) :
        pass
    def handle_arrival_event(self) :
        pass
    def handle_depart_event(self) :
        pass
    def generate_interarrival(self) : 
        pass
    def generate_service(self) : 
        pass
    


def average_time_in_queue( t, lambd, mu ) :
    # This creates an instance of the Simulation class called s
    # by calling the (constructor) function called __init__.
    s = Simulation(lambd,mu)
    # Your code goes here
    
    
    

In [ ]:
runtest(['test_lam'])

# Simulating the queue continued

Now see if you can write a new version the queue code that estimates the distribution for the fraction of time that the queue spends with each possible length that was derived in the first video about queues above.  Remember this video told us that the fraction of time that the queue will have $n$ customers in it is given by:

$$
\pi_n = \left( 1 - \frac{\lambda}{\mu} \right) \left( \frac{\lambda}{\mu}\right)^n
$$

You are not going to use this formula in the next exercise.  You are instead going to estimate the distribution by sampling.  

I have started you off by writing a modified version of the simulation class. I have given you a clue by introducing a variable called `time_with_length` in the class, which I have initialized to a NumPy array with `maxlen` zeros.  Every time you call the `advance_time` function you will need to update an element of this array to reflect the fact that the queue spent a certain amount of time with a particular number of customers within it.  If the number of people in the queue is greater than `maxlen` you will not update any of the elements in this array. 

The other modification I have made is to introduce a function called `get_fraction_of_time_with_length`.  When called this function should return a NumPy array.  The first element of this NumPy array is the fraction of time that there were no people in the queue, the second element is the fraction of time that there is one person in the queue, the third element is the fraction fo time that there are two people in the queue and so on.  You should be able to calculate this array of fractions from the `self.time_with_length` variable and one further variable that was introduced in the video above.

At the end of the cell I have written some code to run a simulation using your `Simulation` class and to plot a graph that shows how long the queue had each particular length.

__The result we obtain from this code was obtained by sampling.  We thus should really quote error bars on this estimate by repeating the simulation multiple times and calculating a variances from these multiple estimates of the histogram if we are presenting our results formally.  If we do not present the errors the result we obtain is not reproducible.__

In [ ]:
class Simulation : 
    def __init__(self, lambd, mu, maxlen) :
        self.lam = lambd
        self.mu = mu
        self.time_with_length = np.zeros(maxlen+1)
        
    def advance_time(self) :
        pass
    def handle_arrival_event(self) :
        pass
    def handle_depart_event(self) :
        pass
    def generate_interarrival(self) : 
        pass
    def generate_service(self) : 
        pass
    
    def get_fraction_of_time_with_length(self):
        pass
    

# This creates an instance of the Simulation class called s
# by calling the (constructor) function called __init__.
lam, expr, N = 0.5, 1, 100000
s = Simulation(lam,expr,8)

# Run a simulation of 10000 events 
for i in range(N) : s.advance_time()

indices, times = np.linspace( 0, 8, 9 ), s.get_fraction_of_time_with_length()
plt.bar( indices, times, width=0.1 )
plt.xlabel("number of people in queue")
plt.ylabel("probability")

# This code is required for the autofeedback- don't delete it!
fighand = plt.gca()

In [ ]:
runtest(['test_variables', 'test_queue'])

# Taking it further

You can present the results that you obtain from writing codes for simulating queues that are presented in this notebook.  However, you should also bear in mind that you have a project for FCT to complete.  I would thus recommend trying to apply the ideas from this notebook to whatever phenomenon that you are trying to simulate in your chosen project.  I think the simulation code that we have written for simulating the queue offers a way of thinking about how to break down a complex phenomenon that are influenced by the interplay of multiple different random events.  For example, it should be possible to extend this code in a way that allows you to consider different types of customers (who require different amounts of time) arriving in a queue.

In short, what I would now really like to see you using the ideas from this notebook creatively.  I want to see if you can apply these ideas to something that you have chosen to simulate. 
